# Transport of NaNO3 in a sand matrix

First of all we did an estimation of what $\dot{Q}$ and $\Delta t$ to set in order to observe the whole process in a relativly short time.
For the $\Delta t$ we settled on 480 seconds  
Measurment time (and therfore measurment points): 1200 seconds

In [6]:
length_column = 19.5
diameter_column = 1
time_before_breakthrough = 5*60 # 5 minutes

required_velocity = length_column / time_before_breakthrough
print(f"Required velocity: {required_velocity} m/s")

Required velocity: 0.065 m/s


In [7]:
flow_area = 3.14 * (diameter_column / 2)**2
column_volume = length_column * flow_area
print(f"Column volume: {column_volume} m^3")

Column volume: 15.307500000000001 m^3


In [8]:
water_mass = 6.524 #grams
water_density = 1 #grams/cm^3
water_volume = water_mass / water_density # cm^3
porosity = water_volume / column_volume # cm^3/cm^3
print(f"Porosity: {porosity}")

Porosity: 0.42619630899885674


In [10]:
real_volume_flow_rate = required_velocity * porosity * flow_area * 60
print(f"Real volume flow rate: {real_volume_flow_rate} cm^3/min = mL/min")

Real volume flow rate: 1.3047999999999997 cm^3/min = mL/min


From this lower limit wr choose to set it to $2 \frac{mL}{min}$ considering that the real flow rate qill be lower than the one the pump is trying to impose since the column and the pipes have some resistance!